# Logistic Regression - multiclass classification

## Read the data 

In [ ]:
import pandas as pd

df_train = pd.read_json("../data/news_train.json", orient="records")
df_train.info()
df_train.hist()
df_train.head()

In [ ]:
df_val = pd.read_json("../data/news_val.json", orient="records")
df_val.info()
df_val.hist()
df_val.head()

In [ ]:
df_test = pd.read_json("../data/news_test.json", orient="records")
df_test.info()
df_test.hist()
df_test.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

STOPWORDS = stopwords.words("english")

def is_html_tag(word):
    w = word.replace("\n", "")
    return w.startswith("<") or w.startswith(">") or w.startswith("/") or w.strip()[:2] == "br"

def remove_html_tags(sentence):
    single_spaces = " ".join(sentence.split())

    return " ".join([token for token in single_spaces.split(" ") if not is_html_tag(token)])

def tokenize_sentence(sentence):
    return word_tokenize(sentence.lower())

def remove_stopwords(sentence):
    return " ".join([word for word in sentence.split(" ") if not word in STOPWORDS])

def remove_punctuation(tokenized_sentence):
    return [word for word in tokenized_sentence if word.isalpha()]

def preprocess(sentence):
    return remove_punctuation(tokenize_sentence(remove_stopwords(remove_html_tags(str(sentence)))))

print(preprocess(df_train["text"].values[1]))


In [ ]:
df_train["text"] = df_train["text"].apply(preprocess)
df_train.head()

In [ ]:
def build_vocab(tokenized_input, vocab_size):
    d = dict()

    for tokens in tokenized_input:
        for token in tokens:
            # double check
            if token not in STOPWORDS and token.isalpha():
                d[token] = d.get(token, 0) + 1

    del d["br"]

    return {k for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)[:vocab_size]}

In [ ]:
VOCAB_SIZE = 10000
VOCAB = build_vocab(df_train["text"].values, VOCAB_SIZE) # (!) Always build the vocabulary on the TRAIN dataset

In [ ]:
len(VOCAB)

In [ ]:
LABELS = df_train["label"].unique()
LABELS

In [ ]:
LABELS_TO_NAMES = {label: df_train[df_train["label"] == label].iloc[0]["label_name"] for label in LABELS}
LABELS_TO_NAMES

In [ ]:
def get_frequencies_for_labels(df):
    dict_freqs = {label: {} for label in LABELS}

    for idx in range(df.shape[0]):
        tokens = df.iloc[idx, 0]
        label = df.iloc[idx, 1]
        
        for token in set(tokens):
            if token in VOCAB:
                dict_freqs[label][token] = dict_freqs[label].get(token, 0) + 1
            

    return dict_freqs

In [ ]:
frequency_table = get_frequencies_for_labels(df_train)

In [ ]:
frequency_table[0]["money"]

In [ ]:
frequency_table[1]["money"]

In [ ]:
import numpy as np

def extract_features(frequency_table, tweet_tokens):
    label_frequencies = {label: 0 for label in LABELS}

    for t in tweet_tokens:
        for label in LABELS:
            label_frequencies[label] += frequency_table[label].get(t, 0)
    
    return pd.Series(label_frequencies)


In [ ]:
X_train_logistic = df_train["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
X_train_logistic

In [ ]:
y_train_logistic = df_train["label"].values
y_train_logistic

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=200).fit(X_train_logistic, y_train_logistic)

In [ ]:
from sklearn.metrics import accuracy_score

preds_train = clf.predict(X_train_logistic)

print("Train accuracy:", accuracy_score(y_train_logistic, preds_train))

## Validation

In [ ]:
X_val = df_val["text"].apply(preprocess)
X_val_logistic = df_val["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
y_val_logistic = df_val["label"].values
X_val_logistic

In [ ]:
preds_val = clf.predict(X_val_logistic)

print("Validation accuracy:", accuracy_score(y_val_logistic, preds_val))

## Test

In [ ]:
X_test = df_test["text"].apply(preprocess)
X_test_logistic = df_test["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
y_test_logistic = df_test["label"].values
X_test_logistic

In [ ]:
preds_test = clf.predict(X_test_logistic)

print("Test accuracy:", accuracy_score(y_test_logistic, preds_test))